In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dense, Lambda
import tensorflow as tf

# Load and preprocess the dataset
data = pd.read_csv('AU.csv')
data = data['Close'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Prepare the training and testing data
def create_dataset(dataset, time_step=1):
    X, Y = [], []
    for i in range(len(dataset) - time_step):
        X.append(dataset[i:(i + time_step), 0])
        Y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(Y)

time_step = 60
X, y = create_dataset(scaled_data, time_step)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Define binomial dropout function
def binomial_dropout(x, n=25, p=0.2):
    mask = tf.zeros_like(x)  # Initialize the mask with zeros
    for _ in range(n):
        dropout_mask = tf.cast(tf.random.uniform(shape=tf.shape(x)) < p, dtype=tf.float32)
        mask += dropout_mask  # Accumulate the dropout mask
    mask = tf.cast(mask <= n*p, dtype=tf.float32)  # Apply binomial condition
    return x * mask

# Create LSTM model with Binomial Dropout
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], 1), return_sequences=True))
model.add(LSTM(50, return_sequences=False))
model.add(Lambda(lambda x: binomial_dropout(x, n=25, p=0.2)))  # Apply binomial dropout
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

# Make predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Invert predictions to original scale
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
y_train = scaler.inverse_transform(y_train.reshape(-1, 1))
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

# Calculate performance metrics
def calculate_metrics(actual, predicted):
    return {
        'R2 Score': r2_score(actual, predicted),
        'MAE': mean_absolute_error(actual, predicted),
        'MSE': mean_squared_error(actual, predicted),
        'RMSE': np.sqrt(mean_squared_error(actual, predicted))
    }

metrics = calculate_metrics(y_test, test_predict)

# Display metrics in a table
metrics_df = pd.DataFrame([metrics], index=['LSTM with binomial Dropout'])
print(metrics_df)



# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(scaler.inverse_transform(scaled_data), label='Actual Prices', color='blue')

# Shift train predictions for plotting
train_predict_plot = np.empty_like(scaled_data)
train_predict_plot[:, :] = np.nan
train_predict_plot[time_step:len(train_predict) + time_step, :] = train_predict

# Shift test predictions for plotting
test_predict_plot = np.empty_like(scaled_data)
test_predict_plot[:, :] = np.nan
# Correct the starting index for the test predictions slice
test_predict_plot[len(train_predict) + time_step:len(train_predict) + time_step + len(test_predict), :] = test_predict

# Plot the predictions
plt.plot(test_predict_plot, label='Test Predictions', color='red')

plt.title('Stock Price Prediction with LSTM and Binomial Dropout')
plt.xlabel('Days')
plt.ylabel('Closing Price')
plt.legend()
plt.show()




2024-08-27 17:57:33.555986: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-27 17:57:33.560146: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-27 17:57:33.577346: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-27 17:57:33.616462: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-27 17:57:33.616524: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-27 17:57:33.638544: I tensorflow/core/platform/cpu_feature_guard.cc:

KeyboardInterrupt: 